# Temporal convergence test

In [ ]:
from pathlib import Path
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests

pd.options.display.max_columns = 5

In [ ]:
headers = {
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
}
print("Get existing data from gist")
response = requests.get(
    f"https://api.github.com/gists/73fa6531f28da2b3633a7ddaca38a7cd",
    headers=headers,
)
data = json.loads(response.json()["files"]["convergence_test.json"]["content"])

In [ ]:
runs = set()
results = defaultdict(list)
spatial_convergence = []
for git_hash, d in data.items():
    runs |= set(d.keys())
    if "dx0.2_dt0.025" not in d:
        # We have not temporaø convergence data
        continue

    for key1, d1 in d.items():
        if "dx0.2" not in key1:
            continue  # Only select data with "dt0.05"
        for key, value in d1.items():
            results[key].append(value)

df = pd.DataFrame(results)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

In [ ]:
df["timestamp"]
df.keys()
df["dt"].unique()

In [ ]:
def get_ylim(values):
    if np.isclose(values, 0).all():
        return (-0.01, 0.01)
    y_mean = np.mean(values)
    y_max = max(np.max(values), y_mean + 0.01 * abs(y_mean))
    y_min = min(np.min(values), y_mean - 0.01 * abs(y_mean))
    return (y_min, y_max)


In [ ]:
columns = [c for c in df.columns if c not in ["timestamp", "simcardems_version", "sha", "dx", "dt"]]
git_hash = df["sha"]
versions = df["simcardems_version"]
dts = df["dt"].unique()
dates = [t.date() for t in df["timestamp"]]


    
fig = make_subplots(rows=len(columns), cols=1, subplot_titles=columns, shared_xaxes=True)

yranges = []
colors = ['red', 'green', 'blue']
for color, dt in zip(colors, sorted(dts)):
    df_dt = df[df["dt"] == dt]
    df_dt.sort_values(by='timestamp')
    
    git_hash = df_dt["sha"]
    versions = df_dt["simcardems_version"]
    dates = [t.date() for t in df_dt["timestamp"]]
    
    text = []
    for h,v, t in zip(git_hash, versions, dates):
        text.append("\n".join([
            f"<br>Git Hash: {h}</br>",
            f"<br>Version: {v}</br>",
            f"<br>Timestamp {t}</br>",
        ]))
 
    for i, c in enumerate(columns):
    
            
    
        row = i + 1
        col = 1
        showlegend=i==0
        fig.add_trace(
            go.Scatter(
                x=df_dt["timestamp"], 
                y=df_dt[c], 
                text=text, 
                hovertemplate="%{text}", 
                name=f"dt={dt}", 
                legendgroup=str(dt),
                showlegend=i==0,
                marker_color=color
            ),
            row=row, col=col
        )
        fig.update_yaxes(row=row, col=col)

fig.update_layout(
    height=7000,
    showlegend=True
)
fig.show()